# H2O GBM Tuning Tutorial for Python

### Arno Candel, PhD, Chief Architect, H2O.ai
### Ported to Python by Navdeep Gill, M.S., Hacker/Data Scientist, H2O.ai

In this tutorial, we show how to build a well-tuned H2O GBM model for a supervised classification task. We specifically don't focus on feature engineering and use a small dataset to allow you to reproduce these results in a few minutes on a laptop. This script can be directly transferred to datasets that are hundreds of GBs large and H2O clusters with dozens of compute nodes.

You can download the source [from H2O's github repository](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb).

Original [R Markdown version here](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.Rmd).

## Installation of the H2O Python Package
Either download H2O from [H2O.ai's website](http://h2o.ai/download) or install the latest version of H2O into Python with the following set of commands:

Install dependencies from command line (prepending with `sudo` if needed):

```
[sudo] pip install -U requests
[sudo] pip install -U tabulate
[sudo] pip install -U future
[sudo] pip install -U six
```

The following command removes the H2O module for Python.
```
[sudo] pip uninstall h2o
```

Next, use pip to install this version of the H2O Python module.
```
[sudo] pip install http://h2o-release.s3.amazonaws.com/h2o/rel-turchin/6/Python/h2o-3.8.2.6-py2.py3-none-any.whl
```

## Launch an H2O cluster on localhost

In [1]:
import h2o
import numpy as np
import math
h2o.init(nthreads=-1, strict_version_check=True)
## optional: connect to a running H2O cluster
#h2o.init(ip="mycluster", port=55555)



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /var/folders/55/rj4cny_s29q4vn1wjt_x08sm0000gn/T/tmpI6TJxH/h2o_navdeepgill_started_from_python.out
JVM stderr: /var/folders/55/rj4cny_s29q4vn1wjt_x08sm0000gn/T/tmpNMvQZm/h2o_navdeepgill_started_from_python.err
Using ice_root: /var/folders/55/rj4cny_s29q4vn1wjt_x08sm0000gn/T/tmplIFvD8


Java Version: java version "1.7.0_79"
Java(TM) SE Runtime Environment (build 1.7.0_79-b15)
Java HotSpot(TM) 64-Bit Server VM (build 24.79-b02, mixed mode)


Starting H2O JVM and connecting: ............ Connection successful!


/Users/navdeepgill/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/navdeepgill/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/navdeepgill/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/navdeepgill/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/navdeepgill/anaconda/lib/python

H2O cluster uptime:,1 seconds 452 milliseconds
H2O cluster version:,3.8.2.6
H2O cluster name:,H2O_started_from_python_navdeepgill_vhl013
H2O cluster total nodes:,1
H2O cluster total free memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


## Import the data into H2O 
Everything is scalable and distributed from now on. All processing is done on the fully multi-threaded and distributed H2O Java-based backend and can be scaled to large datasets on large compute clusters.
Here, we use a small public dataset ([Titanic](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/Titanic.html)), but you can use datasets that are hundreds of GBs large.

In [2]:
## 'path' can point to a local file, hdfs, s3, nfs, Hive, directories, etc.
df = h2o.import_file(path = "http://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")
print df.dim
print df.head
print df.tail
print df.describe

## pick a response for the supervised problem
response = "survived"

## the response variable is an integer, we will turn it into a categorical/factor for binary classification
df[response] = df[response].asfactor()           

## use all other columns (except for the name & the response column ("survived")) as predictors
predictors = df.columns
del predictors[1:3]
print predictors


Parse Progress: [##################################################] 100%
[1309, 14]


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


<bound method H2OFrame.head of >


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


<bound method H2OFrame.tail of >


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


<bound method H2OFrame.describe of >
[u'pclass', u'sex', u'age', u'sibsp', u'parch', u'ticket', u'fare', u'cabin', u'embarked', u'boat', u'body', u'home.dest']


From now on, everything is generic and directly applies to most datasets. We assume that all feature engineering is done at this stage and focus on model tuning. For multi-class problems, you can use `h2o.logloss()` or `h2o.confusion_matrix()` instead of `h2o.auc()` and for regression problems, you can use `h2o.mean_residual_deviance()` or `h2o.mse()`.

## Split the data for Machine Learning
We split the data into three pieces: 60% for training, 20% for validation, 20% for final testing. 
Here, we use random splitting, but this assumes i.i.d. data. If this is not the case (e.g., when events span across multiple rows or data has a time structure), you'll have to sample your data non-randomly.

In [3]:
train, valid, test = df.split_frame(ratios=[0.6,0.2], seed=1234, destination_frames=['train.hex','valid.hex','test.hex'])

## Establish baseline performance
As the first step, we'll build some default models to see what accuracy we can expect. Let's use the [AUC metric](http://mlwiki.org/index.php/ROC_Analysis) for this demo, but you can use `h2o.logloss()` and `stopping_metric="logloss"` as well. It ranges from 0.5 for random models to 1 for perfect models.


The first model is a default GBM, trained on the 60% training split

In [4]:
#We only provide the required parameters, everything else is default
gbm = h2o.H2OGradientBoostingEstimator(distribution='bernoulli')
gbm.train(x=predictors, y=response, training_frame=train)

## Show a detailed model summary
print gbm


gbm Model Build Progress: [##################################################] 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1464886507550_1

Model Summary: 


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,27313.0,5.0,5.0,5.0,10.0,21.0,15.58




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0347141508428
R^2: 0.853514801928
LogLoss: 0.135724386711
AUC: 0.990369609999
Gini: 0.980739219997

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.394257736927: 


,0,1,Error,Rate
0,461.0,18.0,0.0376,(18.0/479.0)
1,13.0,288.0,0.0432,(13.0/301.0)
Total,474.0,306.0,0.0397,(31.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0102564,0.9792763,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0265781,159.1362126,159.1362126
,2,0.0205128,0.9772307,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0531561,159.1362126,159.1362126
,3,0.0307692,0.9749052,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0797342,159.1362126,159.1362126
,4,0.0410256,0.9735284,2.5913621,2.5913621,1.0,1.0,0.0265781,0.1063123,159.1362126,159.1362126
,5,0.0525641,0.9727573,2.5913621,2.5913621,1.0,1.0,0.0299003,0.1362126,159.1362126,159.1362126
,6,0.1,0.9705040,2.5913621,2.5913621,1.0,1.0,0.1229236,0.2591362,159.1362126,159.1362126
,7,0.15,0.9690187,2.5913621,2.5913621,1.0,1.0,0.1295681,0.3887043,159.1362126,159.1362126
,8,0.2,0.9666881,2.5913621,2.5913621,1.0,1.0,0.1295681,0.5182724,159.1362126,159.1362126
,9,0.3,0.9286900,2.5913621,2.5913621,1.0,1.0,0.2591362,0.7774086,159.1362126,159.1362126
,10,0.4,0.3392288,1.8272425,2.4003322,0.7051282,0.9262821,0.1827243,0.9601329,82.7242525,140.0332226




Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-06-02 09:57:06,0.018 sec,0.0,0.2369806,0.6668775,0.5,1.0,0.6141026
,2016-06-02 09:57:07,0.214 sec,1.0,0.2022753,0.5942842,0.9685634,2.5913621,0.0782051
,2016-06-02 09:57:07,0.329 sec,2.0,0.1744836,0.5361788,0.9760888,2.5913621,0.0692308
,2016-06-02 09:57:07,0.437 sec,3.0,0.1520389,0.4883150,0.9760888,2.5913621,0.0692308
,2016-06-02 09:57:07,0.530 sec,4.0,0.1335759,0.4476644,0.9767338,2.5913621,0.0692308
---,---,---,---,---,---,---,---,---
,2016-06-02 09:57:08,1.472 sec,46.0,0.0351867,0.1373187,0.9900159,2.5913621,0.0410256
,2016-06-02 09:57:08,1.490 sec,47.0,0.0350182,0.1368714,0.9901338,2.5913621,0.0410256
,2016-06-02 09:57:08,1.508 sec,48.0,0.0349213,0.1365776,0.9902794,2.5913621,0.0410256
,2016-06-02 09:57:08,1.527 sec,49.0,0.0347513,0.1361386,0.9903765,2.5913621,0.0397436



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
boat,608.0037231,1.0,0.5470799
home.dest,379.0013123,0.6233536,0.3410242
cabin,35.3481979,0.0581381,0.0318062
sex,28.0973225,0.0462124,0.0252819
ticket,21.2186050,0.0348988,0.0190924
embarked,14.9458570,0.0245819,0.0134482
fare,11.1052256,0.0182651,0.0099924
age,5.4856591,0.0090224,0.0049360
parch,3.7122171,0.0061056,0.0033402
sibsp,1.8749361,0.0030838,0.0016871


The AUC is over 94%, so this model is highly predictive!

In [5]:
## Get the AUC on the validation set
perf = gbm.model_performance(valid)
print perf.auc()

0.943195266272


The second model is another default GBM, but trained on 80% of the data (here, we combine the training and validation splits to get more training data), and cross-validated using 4 folds.
Note that cross-validation takes longer and is not usually done for really large datasets.

In [6]:
## rbind() makes a copy here, so it's better to use split_frame with `ratios = c(0.8)` instead above
cv_gbm = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',nfolds = 4, seed = 0xDECAF)
cv_gbm.train(x = predictors, y = response, training_frame = train.rbind(valid))


gbm Model Build Progress: [##################################################] 100%


We see that the cross-validated performance is similar to the validation set performance:

In [7]:
## Show a detailed summary of the cross validation metrics
## This gives you an idea of the variance between the folds
cv_summary = cv_gbm.cross_validation_metrics_summary().as_data_frame()
#print(cv_summary) ## Full summary of all metrics
print(cv_summary.iloc[4]) ## get the row with just the AUCs

## Get the cross-validated AUC by scoring the combined holdout predictions.
## (Instead of taking the average of the metrics across the folds)
#perf_cv = cv_gbm.model_performance(xval=True) ## This only works with H2O version > 3.8.2.6
#print perf_cv.auc()

                     auc
mean            0.943291
sd            0.00774769
cv_1_valid      0.929854
cv_2_valid      0.936152
cv_3_valid       0.95788
cv_4_valid      0.949278
Name: 4, dtype: object


Next, we train a GBM with "I feel lucky" parameters.
We'll use early stopping to automatically tune the number of trees using the validation AUC. 
We'll use a lower learning rate (lower is always better, just takes more trees to converge).
We'll also use stochastic sampling of rows and columns to (hopefully) improve generalization.

In [8]:
gbm_lucky = h2o.H2OGradientBoostingEstimator(
  ## more trees is better if the learning rate is small enough 
  ## here, use "more than enough" trees - we have early stopping
  ntrees = 10000,                                                            

  ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
  learn_rate = 0.01,                                                         

  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 0.8,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 0.8,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 10)

gbm_lucky.train(x=predictors, y=response, training_frame=train, validation_frame=valid)


gbm Model Build Progress: [##################################################] 100%


This model doesn't seem to be much better than the previous models:

In [9]:
perf_lucky = gbm_lucky.model_performance(valid)
print perf_lucky.auc()

0.93933502395


For this small dataset, dropping 20% of observations per tree seems too aggressive in terms of adding regularization. For larger datasets, this is usually not a bad idea. But we'll let this parameter tune freshly below, so no worries.

## Hyper-Parameter Search

Next, we'll do real hyper-parameter optimization to see if we can beat the best AUC so far (around 94%).

The key here is to start tuning some key parameters first (i.e., those that we expect to have the biggest impact on the results). From experience with gradient boosted trees across many datasets, we can state the following "rules":

1. Build as many trees (`ntrees`) as it takes until the validation set error starts increasing.
2. A lower learning rate (`learn_rate`) is generally better, but will require more trees. Using `learn_rate=0.02 `and `learn_rate_annealing=0.995` (reduction of learning rate with each additional tree) can help speed up convergence without sacrificing accuracy too much, and is great to hyper-parameter searches. For faster scans, use values of 0.05 and 0.99 instead.
3. The optimum maximum allowed depth for the trees (`max_depth`) is data dependent, deeper trees take longer to train, especially at depths greater than 10.
4. Row and column sampling (`sample_rate` and `col_sample_rate`) can improve generalization and lead to lower validation and test set errors. Good general values for large datasets are around 0.7 to 0.8 (sampling 70-80 percent of the data) for both parameters. Column sampling per tree (`col_sample_rate_per_tree`) can also be tuned. Note that it is multiplicative with `col_sample_rate`, so setting both parameters to 0.8 results in 64% of columns being considered at any given node to split.
5. For highly imbalanced classification datasets (e.g., fewer buyers than non-buyers), stratified row sampling based on response class membership can help improve predictive accuracy.  It is configured with `sample_rate_per_class` (array of ratios, one per response class in lexicographic order).
6. Most other options only have a small impact on the model performance, but are worth tuning with a Random hyper-parameter search nonetheless, if highest performance is critical.

First we want to know what value of `max_depth` to use because it has a big impact on the model training time and optimal values depend strongly on the dataset.
We'll do a quick Cartesian grid search to get a rough idea of good candidate `max_depth` values. Each model in the grid search will use early stopping to tune the number of trees using the validation set AUC, as before.
We'll use learning rate annealing to speed up convergence without sacrificing too much accuracy.

In [10]:
## Depth 10 is usually plenty of depth for most datasets, but you never know
hyper_params = {'max_depth' : range(1,30,2)}
#hyper_params = {max_depth = c(4,6,8,12,16,20)} ##faster for larger datasets

#Build initial GBM Model
gbm_grid = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ## more trees is better if the learning rate is small enough 
                                    ## here, use "more than enough" trees - we have early stopping
                                    ntrees=10000,
                                    ## smaller learning rate is better
                                    ## since we have learning_rate_annealing, we can afford to start with a 
                                    #bigger learning rate
                                    learn_rate=0.05,
                                    ## sample 80% of rows per tree
                                    sample_rate = 0.8,
                                    ## sample 80% of columns per split
                                    col_sample_rate = 0.8,
                                    ## fix a random number generator seed for reproducibility
                                    seed = 1234,
                                    ## score every 10 trees to make early stopping reproducible 
                                    #(it depends on the scoring interval)
                                    score_tree_interval = 10, 
                                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                                    #5 consecutive scoring events
                                    stopping_rounds = 5,
                                    stopping_metric = "AUC",
                                    stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid = h2o.H2OGridSearch(gbm_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid.train(x=predictors, 
           y=response,
           ## learning rate annealing: learning_rate shrinks by 1% after every tree 
           ## (use 1.00 to disable, but then lower the learning_rate)
           learn_rate_annealing = 0.99, ## this parameter currently cannot be passed above to the constructor
           training_frame = train,
           validation_frame = valid)


gbm Grid Build Progress: [##################################################] 100%


In [11]:
## by default, display the grid search results sorted by increasing logloss (since this is a classification task)
print grid

      max_depth            model_ids   logloss
0            25  depth_grid_model_12  0.215749
1            23  depth_grid_model_11  0.216639
2            29  depth_grid_model_14  0.218503
3            13   depth_grid_model_6  0.218825
4            19   depth_grid_model_9  0.218849
5            27  depth_grid_model_13  0.219126
6            21  depth_grid_model_10  0.219794
7            17   depth_grid_model_8  0.220059
8            11   depth_grid_model_5  0.221247
9             9   depth_grid_model_4  0.222040
10           15   depth_grid_model_7  0.222911
11            7   depth_grid_model_3  0.225722
12            5   depth_grid_model_2  0.232193
13            3   depth_grid_model_1  0.264067
14            1   depth_grid_model_0  0.324154



In [12]:
## sort the grid models by decreasing AUC
sorted_grid = grid.sort_by('auc(valid=True)',increasing=False)
print(sorted_grid)


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,Hyperparameters: [max_depth],auc(valid=True)
depth_grid_model_13,[27],0.9565793
depth_grid_model_12,[25],0.9563539
depth_grid_model_14,[29],0.9562412
depth_grid_model_10,[21],0.9546633
depth_grid_model_9,[19],0.9544942
depth_grid_model_6,[13],0.9543815
depth_grid_model_11,[23],0.9540434
depth_grid_model_5,[11],0.9521837
depth_grid_model_7,[15],0.9517892
depth_grid_model_8,[17],0.9515075


In [13]:
# find the range of the max_depth for the top ten models
top_depths = sorted_grid[0:5]
max_depths = top_depths['Hyperparameters: [max_depth]']
print max_depths

# get the max depths as a list
max_min_list = []
for element in max_depths:
    max_min_list.append(element[0])

([27], [25], [29], [21], [19])


It appears that `max_depth` values of 19 to 29 are best suited for this dataset, which is unusally deep!

In [14]:
new_max = max(max_min_list)
new_min = min(max_min_list)

print "MaxDepth", new_max
print "MinDepth", new_min

MaxDepth 29
MinDepth 19


Now that we know a good range for max_depth, we can tune all other parameters in more detail. Since we don't know what combinations of hyper-parameters will result in the best model, we'll use random hyper-parameter search to "let the machine get luckier than a best guess of any human".

In [15]:
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 600,  ## limit the runtime to 10 minutes
                   'max_models': 1000,  ## build no more than 1000 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }

In [16]:
gbm_final_grid = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ## more trees is better if the learning rate is small enough 
                                    ## here, use "more than enough" trees - we have early stopping
                                    ntrees=10000,
                                    ## smaller learning rate is better
                                    ## since we have learning_rate_annealing, we can afford to start with a 
                                    #bigger learning rate
                                    learn_rate=0.05,
                                    ## score every 10 trees to make early stopping reproducible 
                                    #(it depends on the scoring interval)
                                    score_tree_interval = 10,
                                    ## fix a random number generator seed for reproducibility
                                    seed = 1234,
                                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                                    #5 consecutive scoring events
                                    stopping_rounds = 5,
                                    stopping_metric = "AUC",
                                    stopping_tolerance = 1e-4)
            
#Build grid search with previously made GBM and hyper parameters
final_grid = h2o.H2OGridSearch(gbm_final_grid, hyper_params = hyper_params_tune,
                                    grid_id = 'final_grid',
                                    search_criteria = search_criteria_tune)
#Train grid search
final_grid.train(x=predictors, 
           y=response,
           ## learning rate annealing: learning_rate shrinks by 1% after every tree 
           ## (use 1.00 to disable, but then lower the learning_rate)
           learn_rate_annealing = 0.99,  ## this parameter currently cannot be passed above to the constructor
           ## early stopping based on timeout (no model should take more than 1 hour - modify as needed)
           max_runtime_secs = 3600, 
           training_frame = train,
           validation_frame = valid)

print final_grid


gbm Grid Build Progress: [##################################################] 100%
         histogram_type  nbins_cats  sample_rate  nbins  min_rows  \
0            RoundRobin          16         1.00   1024         4   
1       UniformAdaptive          64         0.93   1024         4   
2            RoundRobin          32         0.40   1024        16   
3       QuantilesGlobal          64         0.79     64         4   
4       UniformAdaptive          32         0.29    256         4   
5       UniformAdaptive          64         0.73     32         8   
6       QuantilesGlobal         128         0.42     64         4   
7       UniformAdaptive         256         0.55    128         2   
8            RoundRobin          16         0.88     16         4   
9            RoundRobin          64         0.74   1024        32   
10      UniformAdaptive          32         0.34    128         2   
11      UniformAdaptive         128         0.52     64         8   
12           RoundR

We can see that the best models have even better validation AUCs than our previous best models, so the random grid search was successful!

In [17]:
## Sort the grid models by AUC
sorted_final_grid = final_grid.sort_by('auc(valid=True)',increasing=False)
print sorted_final_grid


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [nbins, col_sample_rate, min_split_improvement, col_sample_rate_per_tree, min_rows, col_sample_rate_change_per_level, nbins_cats, sample_rate, histogram_type, max_depth]",auc(valid=True)
final_grid_model_93,"[1024, 0.37, 0.0001, 0.94, 4.0, 1.05, 16, 1.0, u'RoundRobin', 20]",0.9742744
final_grid_model_177,"[1024, 0.31, 1e-06, 0.96, 4.0, 1.03, 64, 0.93, u'UniformAdaptive', 20]",0.9724711
final_grid_model_68,"[64, 0.38, 0.0001, 0.96, 4.0, 1.06, 64, 0.79, u'QuantilesGlobal', 26]",0.9697661
final_grid_model_65,"[16, 0.52, 0.0, 0.51, 1.0, 1.1, 256, 0.86, u'RoundRobin', 24]",0.9692589
final_grid_model_214,"[64, 0.87, 0.0001, 0.66, 4.0, 1.05, 128, 0.42, u'QuantilesGlobal', 21]",0.9692026
---,---,---
final_grid_model_48,"[1024, 0.27, 0.0, 0.38, 256.0, 1.05, 256, 0.63, u'UniformAdaptive', 29]",0.7787264
final_grid_model_207,"[1024, 0.32, 1e-06, 1.0, 256.0, 1.0, 512, 0.92, u'QuantilesGlobal', 29]",0.7513948
final_grid_model_8,"[64, 0.95, 0.0, 0.59, 256.0, 1.09, 4096, 0.24, u'RoundRobin', 28]",0.7507749
final_grid_model_75,"[512, 0.64, 1e-06, 0.99, 256.0, 1.0, 512, 0.49, u'RoundRobin', 24]",0.7027050



See the whole table with table.as_data_frame()



We can inspect the best 5 models from the grid search explicitly, and query their validation AUC:

In [18]:
print sorted_final_grid["auc(valid=True)"][0:5]

(0.9742744435052128, 0.9724711186249648, 0.9697661313045929, 0.9692589461820231, 0.9692025922795153)


You can also see the results of the grid search in [Flow](http://localhost:54321/):
![alt text](./final_grid.png "Final Grid Search Results in Flow")

## Model Inspection and Final Test Set Scoring

Let's see how well the best model of the grid search (as judged by validation set AUC) does on the held out test set:

In [19]:
#Get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model('final_grid_model_46')
performance_best_model = best_model.model_performance(test)

Good news. It does as well on the test set as on the validation set, so it looks like our best GBM model generalizes well to the unseen test set:

In [20]:
#Get the performance on the test model
print performance_best_model.auc()

0.965706356548


We can inspect the winning model's parameters:

In [21]:
for key, value in best_model.params.iteritems():
    print key,value['actual']

learn_rate 0.05
fold_column None
col_sample_rate_per_tree 1.0
learn_rate_annealing 0.99
score_tree_interval 10
sample_rate_per_class None
seed 1234
keep_cross_validation_predictions False
model_id {u'URL': u'/4/Models/final_grid_model_46', u'type': u'Key<Model>', u'name': u'final_grid_model_46', u'__meta': {u'schema_name': u'ModelKeyV3', u'schema_version': 3, u'schema_type': u'Key<Model>'}}
nfolds 0
max_abs_leafnode_pred 1.79769313486e+308
offset_column None
quantile_alpha 0.5
stopping_tolerance 0.0001
fold_assignment AUTO
training_frame {u'URL': u'/4/Frames/train.hex', u'type': u'Key<Frame>', u'name': u'train.hex', u'__meta': {u'schema_name': u'FrameKeyV3', u'schema_version': 3, u'schema_type': u'Key<Frame>'}}
max_runtime_secs 469.338
checkpoint None
balance_classes False
r2_stopping 0.999999
validation_frame {u'URL': u'/4/Frames/valid.hex', u'type': u'Key<Frame>', u'name': u'valid.hex', u'__meta': {u'schema_name': u'FrameKeyV3', u'schema_version': 3, u'schema_type': u'Key<Frame>'}}
m

Now we can confirm that these parameters are generally sound, by building a GBM model on the whole dataset (instead of the 60%) and using internal 5-fold cross-validation (re-using all other parameters including the seed):

In [22]:
gbm_best = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
                                    ntrees=10000,
                                    learn_rate=0.05,
                                    col_sample_rate = 0.86,
                                    col_sample_rate_change_per_level = 1.07,
                                    col_sample_rate_per_tree = 1,
                                    seed = 1234,
                                    sample_rate = 0.39,
                                    score_tree_interval = 10, 
                                    stopping_rounds = 5,
                                    stopping_metric = "AUC",
                                    stopping_tolerance = 1e-4,
                                    nbins_cats = 32,
                                    nbins = 128,
                                    min_rows = 8,
                                    max_depth = 26,
                                    histogram_type = "QuantilesGlobal",
                                    min_split_improvement = 1e-6,
                                    nfolds = 5)

In [23]:
gbm_best.train(x=predictors, y=response,learn_rate_annealing = 0.99, training_frame=df)


gbm Model Build Progress: [##################################################] 100%


In [24]:
print gbm_best.cross_validation_metrics_summary()


Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
F0point5,0.9151056,0.0143311,0.9255725,0.8898305,0.8921162,0.9395973,0.9284117
F1,0.896728,0.0087329,0.9023256,0.8842105,0.8865979,0.9180328,0.8924731
F2,0.8793351,0.0085851,0.8802178,0.8786611,0.8811476,0.8974359,0.8592132
accuracy,0.9237242,0.0063880,0.9213483,0.9188192,0.9147287,0.9409449,0.9227799
auc,0.9597895,0.0059350,0.9579493,0.9649405,0.9552934,0.9726580,0.9481061
err,0.0762758,0.0063880,0.0786517,0.0811808,0.0852713,0.0590551,0.0772201
err_count,20.0,1.8439089,21.0,22.0,22.0,15.0,20.0
lift_top_group,2.6258688,0.0998947,2.3839285,2.8229167,2.632653,2.6736841,2.6161616
logloss,0.2383605,0.0212559,0.2668741,0.2454505,0.2632381,0.1835541,0.2326858
max_per_class_error,0.1317566,0.0113294,0.1339286,0.125,0.1224490,0.1157895,0.1616162


Ouch! So it looks like we overfit quite a bit on the validation set as the mean AUC on the 5 folds is "only" 96.28%, with a standard deviation of 0.6% (absolute). So we cannot always expect AUCs of 97% with these parameters on this dataset. So to get a better estimate of model performance, the Random hyper-parameter search could have used `nfolds = 5` (or 10, or similar) in combination with 80% of the data for training (i.e., not holding out a validation set, but only the final test set). However, this would take more time, as `nfolds+1` models will be built for every set of parameters.

Another idea is to test the performance of the top 5 models via 5-fold cross-validation and pick the best one, or make an ensemble. See the [R Markdown](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.Rmd) version of this script for that.

Keeping the same "best" model, we can make test set predictions as follows:

In [25]:
preds = best_model.predict(test)
preds.head()
#gbm@model$validation_metrics@metrics$max_criteria_and_metric_scores


gbm prediction Progress: [##################################################] 100%


predict,p0,p1
0,0.978769,0.0212309
0,0.991286,0.00871367
0,0.832473,0.167527
1,0.0120367,0.987963
1,0.00909804,0.990902
0,0.827661,0.172339
1,0.0226573,0.977343
1,0.0111063,0.988894
1,0.0223358,0.977664
0,0.9004,0.0996002


Note that the label (survived or not) is predicted as well (in the first predict column), and it uses the threshold with the highest F1 score (here: 0.5432907) to make labels from the probabilities for survival (`p1`). The probability for death (`p0`) is given for convenience, as it is just `1-p1`.

In [26]:
best_model.model_performance()


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0429931655609
R^2: 0.818579391401
LogLoss: 0.155110901234
AUC: 0.991898958933
Gini: 0.983797917866

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.267658143171: 


,0,1,Error,Rate
0,445.0,34.0,0.071,(34.0/479.0)
1,6.0,295.0,0.0199,(6.0/301.0)
Total,451.0,329.0,0.0513,(40.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2676581,0.9365079,178.0
max f2,0.2676581,0.9621657,178.0
max f0point5,0.5466314,0.9613929,113.0
max accuracy,0.3348736,0.95,157.0
max precision,0.9925535,1.0,0.0
max recall,0.0806368,1.0,275.0
max specificity,0.9925535,1.0,0.0
max absolute_MCC,0.2676581,0.8961266,178.0
max min_per_class_accuracy,0.3118373,0.9436326,164.0



Gains/Lift Table: Avg response rate: 38.59 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0102564,0.9912775,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0265781,159.1362126,159.1362126
,2,0.0205128,0.9905533,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0531561,159.1362126,159.1362126
,3,0.0307692,0.9901541,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0797342,159.1362126,159.1362126
,4,0.0410256,0.9899583,2.5913621,2.5913621,1.0,1.0,0.0265781,0.1063123,159.1362126,159.1362126
,5,0.05,0.9896410,2.5913621,2.5913621,1.0,1.0,0.0232558,0.1295681,159.1362126,159.1362126
,6,0.1,0.9873104,2.5913621,2.5913621,1.0,1.0,0.1295681,0.2591362,159.1362126,159.1362126
,7,0.15,0.9841872,2.5913621,2.5913621,1.0,1.0,0.1295681,0.3887043,159.1362126,159.1362126
,8,0.2,0.9783290,2.5913621,2.5913621,1.0,1.0,0.1295681,0.5182724,159.1362126,159.1362126
,9,0.3012821,0.6755902,2.5585601,2.5803351,0.9873418,0.9957447,0.2591362,0.7774086,155.8560074,158.0335053
,10,0.4,0.3112699,1.7163567,2.3671096,0.6623377,0.9134615,0.1694352,0.9468439,71.6356733,136.7109635


You can also see the "best" model in more detail in [Flow](http://localhost:54321/):
![alt text](./best_gbm1.png "Best GBM from Grid Search in Flow")
![alt text](./best_gbm2.png "Best GBM from Grid Search in Flow")

The model and the predictions can be saved to file as follows:

In [95]:
h2o.save_model(best_model, "/tmp/bestModel.csv", force=True)
h2o.export_file(preds, "/tmp/bestPreds.csv", force=True)


Export File Progress: [##################################################] 100%


In [96]:
#h2o.download_pojo(best_model)

The model can also be exported as a plain old Java object (POJO) for H2O-independent (standalone/Storm/Kafka/UDF) scoring in any Java environment.

```
/*
  Licensed under the Apache License, Version 2.0
    http://www.apache.org/licenses/LICENSE-2.0.html

  AUTOGENERATED BY H2O at 2016-05-27T22:11:18.485-07:00
  3.8.2.6
  
  Standalone prediction code with sample test data for GBMModel named final_grid_model_46

  How to download, compile and execute:
      mkdir tmpdir
      cd tmpdir
      curl http://127.0.0.1:54321/3/h2o-genmodel.jar > h2o-genmodel.jar
      curl http://127.0.0.1:54321/3/Models.java/final_grid_model_46 > final_grid_model_46.java
      javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m final_grid_model_46.java

     (Note:  Try java argument -XX:+PrintCompilation to show runtime JIT compiler behavior.)
*/
...
class final_grid_model_46_Tree_289_class_0 {
  static final double score0(double[] data) {
    double pred =      (!Double.isNaN(data[9]) && data[9 /* boat */] != 12.0f ? 
         (data[9 /* boat */] <13.003977f ? 
             (data[9 /* boat */] <11.504464f ? 
                 (data[1 /* sex */] <0.5f ? 
                     (data[6 /* fare */] <24.978184f ? 
                        0.0027945167f : 
                         (data[11 /* home.dest */] <132.0f ? 
                            0.0027875851f : 
                             (data[11 /* home.dest */] <265.5f ? 
                                 (data[6 /* fare */] <49.690052f ? 
                                    0.002784563f : 
                                    0.0027828787f) : 
                                0.0027856496f))) : 
                    0.002798641f) : 
                0.0028186426f) : 
            0.00350058f) : 
...
```

## Ensembling Techniques

After learning above that the variance of the test set AUC of the top few models was rather large, we might be able to turn this into our advantage by using ensembling techniques. The simplest one is taking the average of the predictions (survival probabilities) of the top `k` grid search model predictions (here, we use `k=10`):

In [33]:
prob = None
k=10
for i in range(0,k): 
    gbm = h2o.get_model(sorted_final_grid["Model Id"][i])
    if (prob is None):
        prob = gbm.predict(test)["p1"]
    else:
        prob = prob + gbm.predict(test)["p1"]
prob = prob/k


gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%


We now have a blended probability of survival for each person on the Titanic.

In [34]:
prob.head()

p1
0.0270728
0.0163243
0.139536
0.981392
0.984146
0.164179
0.961535
0.981923
0.961027
0.106759


We can bring those ensemble predictions to our Python session's memory space and use other Python packages.

This simple blended ensemble test set prediction has an even higher AUC than the best single model, so we can assume that this would hold true using cross-validation as well. We leave this as an exercise for the reader - take the parameters of the top `10` models, retrain them with `nfolds=5` on the full dataset, set `keep_holdout_predictions=TRUE` and sum up their predicted probabilities in `h2o.getFrame(cvgbm[i]@model$cross_validation_holdout_predictions_frame_id)`, then score that with cvAUC as shown above).

For more sophisticated ensembling approaches, such as stacking via a superlearner, we refer to the [H2O Ensemble](https://github.com/h2oai/h2o-3/tree/master/h2o-r/ensemble) github page.

## Summary
We learned how to build H2O GBM models for a binary classification task on a small but realistic dataset with numerical and categorical variables, with the goal to maximize the AUC (ranges from 0.5 to 1). We first established a baseline with the default model, then carefully tuned the remaining hyper-parameters without "too much" human guess-work. We used both Cartesian and Random hyper-parameter searches to find good models. We were able to get the AUC on a holdout test set from the low 94% range with the default model to the mid 97% after tuning, and to the high 97% with some simple ensembling technique known as blending. We performed simple cross-validation variance analysis to learn that results were slightly "lucky" due to the specific train/valid/test set splits, and settled to expect mid 96% AUCs instead.

Note that this script and the findings therein are directly transferrable to large datasets on distributed clusters including Spark/Hadoop environments.

More information can be found here [http://www.h2o.ai/docs/](http://www.h2o.ai/docs/).